# 문자 단위 RNN (char RNN)
모든 시점의 입력에 대해서 출력하는 many-to-many RNN 구현

## 문자 단위 RNN(Char RNN)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### 데이터 전처리

In [2]:
input_str = 'apple'
label_str = 'pple!'
print(set(input_str+label_str))
char_vocab = sorted(list(set(input_str+label_str)))
print(char_vocab)
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

{'e', 'l', '!', 'p', 'a'}
['!', 'a', 'e', 'l', 'p']
문자 집합의 크기 : 5


In [3]:
## 문자마다 고유한 정수 부여
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 정수 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [4]:
## 정수를 문자로 바꾸는 key value 설정
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [5]:
## 입력 데이터와 레이블 데이터를 정수로 맵핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


입력 시퀀스의 각 문자들을 원-핫 벡터로 바꿈

In [6]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([0., 1., 0., 0., 0.]), array([0., 0., 0., 0., 1.]), array([0., 0., 0., 0., 1.]), array([0., 0., 0., 1., 0.]), array([0., 0., 1., 0., 0.])]


In [7]:
X = torch.FloatTensor(x_one_hot).unsqueeze(0)
Y = torch.LongTensor(y_data)

print(X)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

tensor([[[0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.]]])
훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([5])


### 하이퍼 파라미터

In [8]:
input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

### 모델 구현

In [9]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
    
    def forward(self,x):# 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x
    
net = Net(input_size, hidden_size, output_size)


In [10]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서 : 배치 차원, 시점(timesteps), 출력의 크기

torch.Size([1, 5, 5])


나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산 => view를 사용하여 배치 차원과 시점 차원을 하나로 만듬

In [11]:
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

torch.Size([5, 5])


In [12]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([5])
torch.Size([5])


레이블 데이터는 (1, 5)의 크기를 가지는데, 마찬가지로 나중에 정확도를 측정할 때는 이걸 펼쳐서 계산   
이 경우 (5)의 크기를 가지게 됨

### 손실함수 & optimizer

In [13]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

### 학습
- 100 에폭 학습

In [14]:
net.train()

for i in range(10):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.705937385559082 prediction:  [[2 2 4 2 2]] true Y:  [4, 4, 3, 2, 0] prediction str:  eepee
1 loss:  1.348324179649353 prediction:  [[4 4 4 2 4]] true Y:  [4, 4, 3, 2, 0] prediction str:  pppep
2 loss:  1.1227716207504272 prediction:  [[4 4 4 2 4]] true Y:  [4, 4, 3, 2, 0] prediction str:  pppep
3 loss:  0.9485987424850464 prediction:  [[4 4 4 2 4]] true Y:  [4, 4, 3, 2, 0] prediction str:  pppep
4 loss:  0.7764447927474976 prediction:  [[4 4 3 2 4]] true Y:  [4, 4, 3, 2, 0] prediction str:  pplep
5 loss:  0.5973604321479797 prediction:  [[4 4 3 2 4]] true Y:  [4, 4, 3, 2, 0] prediction str:  pplep
6 loss:  0.4496040344238281 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
7 loss:  0.3524056077003479 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
8 loss:  0.2669774889945984 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2, 0] prediction str:  pple!
9 loss:  0.1952563226222992 prediction:  [[4 4 3 2 0]] true Y:  [4, 4, 3, 2